In [1]:
import warnings

warnings.filterwarnings("ignore")

import math
import os
import re
import sys
from collections import Counter
from copy import deepcopy
from functools import reduce
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from rich import print
from tqdm import tqdm

In [2]:
def plot_mtx(mtx):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.matshow(mtx, cmap=plt.cm.Blues)
    for i in range(mtx.shape[0]):
        for j in range(mtx.shape[1]):
            c = mtx[i, j]
    plt.plot()

In [3]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """Convert a list of string to a list of other types

    pattern: a list of types for one item.
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [4]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

# head = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=True, sep=",", start_row=0, end_row=1
# )

# print(f"Head:\n{head}")
# print(f"First line: {head[0]}")
# print(f"Last line: {head[-1]}")

data = convert_str_li_to_other_li(
    lines, pattern="s", per_letter=False, sep=" ", start_row=None, end_row=None
)


print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: Monkey 0: | Last line:     If false: throw to monkey 3

------

Convert to:

First line: ['Monkey', '0:']

Last line: ['If', 'false:', 'throw', 'to', 'monkey', '3']

------

In [5]:
monkeys = dict()
monkey_ops = dict()
monkey_div = dict()
monkey_throw = dict()
current_monkey = -1
for l in data:
    if len(l) == 0:
        continue
    if l[0] == "Monkey":
        current_monkey = int(l[1].split(":")[0])
    elif l[0] == "Starting":
        monkeys[current_monkey] = [
            int(j) for j in "".join([l[i] for i in range(2, len(l))]).split(",")
        ]
    elif l[0] == "Operation:":
        monkey_ops[current_monkey] = " ".join(l[3:])
    elif l[0] == "Test:":
        monkey_div[current_monkey] = int(l[-1])
    elif l[0] == "If":
        if l[1] == "true:":
            monkey_throw[current_monkey] = [
                int(l[-1]),
            ]
        else:
            monkey_throw[current_monkey].append(int(l[-1]))

mul = 1
for i in monkey_div.values():
    mul *= i

monkeys_org = deepcopy(monkeys)

In [6]:
%%time
for q in [1, 2]:
    total_round = 20 if q == 1 else 10_000
    monkeys = deepcopy(monkeys_org)
    count_ins = {n: 0 for n in range(len(monkeys))}
    for r in range(total_round):
        for n in range(len(monkeys)):
            for item in monkeys[n]:
                count_ins[n] += 1
                if q == 1:
                    new_item = eval(monkey_ops[n].replace("old", str(item))) // 3
                else:
                    new_item = eval(monkey_ops[n].replace("old", str(item))) % mul
                if new_item % monkey_div[n] == 0:
                    to_n = monkey_throw[n][0]
                else:
                    to_n = monkey_throw[n][1]
                monkeys[to_n].append(new_item)
            monkeys[n] = []

    count_li = list(count_ins.values())
    count_li.sort()
    print(f"Answer to Q{q}: {count_li[-1] * count_li[-2]}")

Answer to Q1: 99840

Answer to Q2: 20683044837

CPU times: user 1.69 s, sys: 10.1 ms, total: 1.7 s
Wall time: 1.7 s
